<a href="https://colab.research.google.com/github/DariuszKobiela/DigitPredictorAI/blob/master/NN_GOOGLE_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow
tensorflow.__version__

'2.2.0-rc3'

# IMPORTS

In [0]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.metrics import Accuracy, Recall, Precision
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical, plot_model


def load_data_for_NN(root_directory):
    mnist_train = pd.read_csv(root_directory+"data\\mnist_train.csv")
    mnist_test = pd.read_csv(root_directory+"data\\mnist_test.csv")
    y_train = mnist_train.label.values
    y_test = mnist_test.label.values
    x_train = mnist_train.values[:, 1:]
    x_test = mnist_test.values[:, 1:]
    # Standardization
    x_train = np.array([(x - x.mean()) / x.std() for x in x_train])
    x_test = np.array([(x - x.mean()) / x.std() for x in x_test])
    x_train = x_train.astype('float32').reshape(-1, 28, 28, 1)
    x_test = x_test.astype('float32').reshape(-1, 28, 28, 1)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return x_train, y_train, x_test, y_test


def load_images(root_directory):
    mnist = pd.read_csv(root_directory+"data\\mnist_test.csv")
    y = mnist.label.values
    x = mnist.values[:, 1:].astype('float32').reshape(-1, 28, 28)
    return x, y


def model_DNN():
    model = Sequential()
    model.add(Flatten(input_shape=[28, 28, 1]))
    model.add(Dense(512))
    model.add(Dense(128))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model


def model_CNN():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[28, 28, 1]))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc'])
    return model


def get_checkpoint_path(model_name, root_directory):
    checkpoint_path = root_directory + model_name + "_training/cp.ckpt"
    return checkpoint_path


def fit_model(model, model_name, x_train, y_train, batch_size, num_epoch, root_directory):
    checkpoint_path = get_checkpoint_path(model_name, root_directory)
    cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                  save_weights_only=True,
                                  verbose=1,
                                  monitor="loss")
    model_log = model.fit(x_train, y_train, batch_size=batch_size, validation_split=0.3, epochs=num_epoch,
                          verbose=1, callbacks=[cp_callback])
    return model_log, model


def validate(model, x_train, y_train, x_test, y_test):
    score = model.evaluate(x_train, y_train, verbose=0)
    print('Train loss:', score[0], end=", ")
    print('Train accuracy:', score[1])
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0], end=", ")
    print('Test accuracy:', score[1])
    print(model.summary())


def save_model_plot(model, model_name, root_directory):
    plot_model(model, to_file=root_directory+"images\\" + model_name + '_architecture.png', show_shapes=True, show_layer_names=False)


def plot_metric(model_log, metric_name, model_name, root_directory):
    fig = plt.figure()
    fig.set_size_inches(6, 4)
    plt.plot(model_log.history[metric_name])
    plt.plot(model_log.history['val_' + metric_name])
    plt.title('model' + model_name + " - " + metric_name)
    plt.ylabel(metric_name)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='lower right')
    plt.savefig(root_directory + "images\\" + model_name + '_' + metric_name + '.png')


# TRAINING

In [0]:
batch_size = 128
num_epoch = 3
root_directory = "/content/drive/My Drive/COLAB/"

print("--------------------------------")

x_train, y_train, x_test, y_test = load_data_for_NN(root_directory)

for model_name, model in [("CNN", model_CNN()), ("DNN", model_DNN())]:
    model_log, model = fit_model(model, model_name, x_train, y_train, batch_size, num_epoch, root_directory)
    validate(model, x_train, y_train, x_test, y_test)
    save_model_plot(model, model_name, root_directory)
    for metric_name in ['accuracy', 'loss']:
        plot_metric(model_log, metric_name, model_name, root_directory)

In [6]:
root_directory = "/content/drive/My Drive/COLAB/"

x_train, y_train, x_test, y_test = load_data_for_NN(root_directory)

FileNotFoundError: ignored

# LOADING

In [0]:
# model_name = "CNN"
# checkpoint_path = get_checkpoint_path(model_name, root_directory)
# checkpoint_dir = os.path.dirname(checkpoint_path)
#
# model = model_CNN()
# latest = tf.train.latest_checkpoint(checkpoint_dir)
# model.load_weights(latest)

# print(x_test[0].shape)
#     # print(model.predict(x_test[0].reshape(1, 28, 28, 1)))